In [22]:
import yaml

In [23]:
subnet_list = ['User',
              'Enterprise',
              'Operational_A',
               'Operational_B',
              ]

subnet_host_types = ['User','Enterprise','Op_Server','Op_Host']

image_map = {
    'User': 'Internal', # Windows
    'Enterprise': 'Internal', # Windows
    'Op_Server': 'Gateway', # Linux
    'Op_Host': 'Gateway', # Linux
}

subnet_sizes = [2,3,1,2]

subnet_data = zip(subnet_list,subnet_host_types,subnet_sizes)

In [24]:
subnets = {}
hosts = {}

for name, host_type, number in subnet_data:
    host_list = []
    for i in range(number):
        host_list.append(host_type+str(i))
        hosts[host_list[-1]] = {'image': image_map[host_type],
                              'AWS_Info': [],
                             }
        subnets[name] = {
            'Hosts':host_list,
            'Size':number,
            'NACLs':{'all':{'in':'all','out':'all'}}
        }

In [25]:
red_actions = ['Sleep',
               'DiscoverRemoteSystem',
               'DiscoverNetworkService',
               'DiscoverRemoteVulnerability',
               'Exploit',
               'Escalate',
               'Impact',
              ]

red_sessions = [{'username':'SYSTEM',
               'hostname':'User0',
               'type':'MetasploitServer',
               'name':'MSFServer',
              }]

red_int = {'Hosts':{'User1':
                    {'Interfaces':['User'],
                     'System info': 'All',
                    }
                   }
          }

red = {'agent_type':'SleepAgent',
        'wrappers':[],
        'actions':red_actions,
        'starting_sessions':red_sessions,
        'reward_calculator_type':'Pwn',
        'INT': red_int,
        'AllowedSubnets': ['User'],
      }

In [26]:
blue_actions = ['Sleep',
               'Monitor',
               'Analyse',
               'Block',
               'Unblock',
               'Restore',
               'Deceive',
              ]

blue_sessions = [{'username':'ubuntu',
                'hostname':'Enterprise1',
                'type':'VelociraptorServer',
                'name':'VeloServer',
                'num_children_sessions':2,
                 'artifacts': ['Generic.Client.Stats',
                              'Linux.Events.SSHLogin',
                              'Windows.Events.ProcessCreation',
                              ]
              },
                {'name':'VeloInternal',
                'username':'SYSTEM',
                'type':'VelociraptorClient',
                'hostname':'Internal',
                'parent':'VeloServer'
                },
                {'name':'VeloGateway',
                'username':'ubuntu',
                'type':'VelociraptorClient',
                'hostname':'Gateway',
                'parent':'VeloServer'
                }]

blue_host_dict = {}
for host in hosts.keys():
    blue_host_dict[host] = {
        'Interfaces':[host[0:-1]], #More than ten on subnet it breaks
        'Creds':'All',
        'OS_info':{},
        'System info': 'All',
    }

blue_int = {'Hosts':blue_host_dict}

blue = {'agent_type':'SleepAgent',
        'wrappers':[],
        'actions':blue_actions,
        'starting_sessions':blue_sessions,
        'reward_calculator_type':'SimpleAvailability',
        'INT': red_int,
        'AllowedSubnets': ['User',
                           'Enterprise',
                           'Op_Server',
                           'Op_Host',
                          ]
      }

green_actions = ['Sleep',
                'AddFile',
                'ModifyFile',
                'DeleteFile',
                'DownloadFile',
                'MoveFile',
                'CreateProcess',
                'RemoveProcess',
                'GreenPortScan',
                'GreenPingSweep',
                'GreenSSHConnection',
                'MagicOTConnection'
               ]

green_sessions = []

for host in hosts.keys():
    sess = {'username':'GreenAgent',
                'hostname':host,
                'type':'Session',
                'name':'GreenSession',
           }
    green_sessions.append(sess)

green = {'agent_type':'SleepAgent',
        'wrappers':[],
        'actions':green_actions,
        'starting_sessions':green_sessions,
        'reward_calculator_type':[],
        'INT': [],
        'AllowedSubnets': ['User',
                           'Enterprise',
                           'Op_Server',
                           'Op_Host',
                          ]
      }

agents = {'Red':red,
          'Blue':blue,
         'Green':green
         }

In [27]:
scenario = {'Agents':agents,
           'Subnets':subnets,
           'Hosts':hosts}

In [28]:
output = yaml.dump(scenario)

In [29]:
print(output)

Agents:
  Blue:
    AllowedSubnets:
    - User
    - Enterprise
    - Op_Server
    - Op_Host
    INT: &id001
      Hosts:
        User1:
          Interfaces:
          - User
          System info: All
    actions:
    - Sleep
    - Monitor
    - Analyse
    - Block
    - Unblock
    - Restore
    - Deceive
    agent_type: SleepAgent
    reward_calculator_type: SimpleAvailability
    starting_sessions:
    - artifacts:
      - Generic.Client.Stats
      - Linux.Events.SSHLogin
      - Windows.Events.ProcessCreation
      hostname: Enterprise1
      name: VeloServer
      num_children_sessions: 2
      type: VelociraptorServer
      username: ubuntu
    - hostname: Internal
      name: VeloInternal
      parent: VeloServer
      type: VelociraptorClient
      username: SYSTEM
    - hostname: Gateway
      name: VeloGateway
      parent: VeloServer
      type: VelociraptorClient
      username: ubuntu
    wrappers: []
  Green:
    AllowedSubnets:
    - User
    - Enterprise
    - Op_Se

In [30]:
with open('Scenario2_trial.yaml','w') as file:
    yaml.dump(scenario,file)